In [1]:
from code_tokenizer import *

code = """
json = {
    "city": "New York"
    "name": "John",
    "age": 30,
}
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""
print(color_by_char_type(code))
print(parse(code))


json = {
    "city": "New York"
    "name": "John",
    "age": 30,
}
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

[['word', 'json'], ['symbol', '='], ['{', [['"', [['word', 'city']]], ['symbol', ':'], ['"', [['word', 'New'], ['word', 'York']]], ['"', [['word', 'name']]], ['symbol', ':'], ['"', [['word', 'John']]], ['symbol', ','], ['"', [['word', 'age']]], ['symbol', ':'], ['number', '30'], ['symbol', ',']]], ['word', 'my_list'], ['symbol', '='], ['[', [['number', '1'], ['symbol', ','], ['number', '2'], ['symbol', ','], ['number', '3'], ['symbol', ','], ['number', '4'], ['symbol', ','], ['number', '5']]], ['word', 'min'], ['symbol', '='], ['word', 'my_list'], ['[', [['number', '0']]], ['word', 'for'], ['(', [['word', 'i'], ['symbol', '='], ['number', '1'], ['symbol', ';'], ['word', 'i'], ['symbol', '<'], ['word', 'my_list'], ['symbol', '.'], ['word', 'length'], ['symbol', 